<a href="https://colab.research.google.com/github/DACAX22/TrabajoFinal/blob/main/Trabajo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from numpy.core.fromnumeric import shape
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier as KNc
from sklearn.svm import SVC as svc
from sklearn.ensemble import RandomForestClassifier as RFc
from sklearn.ensemble import GradientBoostingClassifier as GBC
from sklearn.model_selection import train_test_split as tts
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, recall_score, precision_score
from matplotlib.pylab import subplots
import plotly.express as px


weather_df=pd.read_csv('/content/seattle-weather.csv')
print(weather_df)
print('')

def usuario_dec():
  dec=input("Si desea hacer una acción escriba W: ")
  if dec=="W":
    estado=1
  else:
    estado=2
  return estado

#Conversión data
x_new=weather_df.drop(columns={'date'})

x_weather=x_new['weather']
x_weatherarr=np.array(x_weather)

encoder=LabelEncoder()
x_weatherencoder=encoder.fit_transform(x_weatherarr)

weather_tra=x_new.drop(columns=['weather'])
weather_tra['weather']=x_weatherencoder

precipitation_array=np.array(weather_tra['precipitation'])
temmax_array=np.array(weather_tra['temp_max'])
temmin_array=np.array(weather_tra['temp_min'])
wind_array=np.array(weather_tra['wind'])
X_=np.c_[precipitation_array,temmax_array,temmin_array,wind_array]
Y_=np.array(weather_tra['weather'])
Y_= np.reshape(Y_,(len(Y_),1))

#Escalamos
scaler = MinMaxScaler(feature_range=(0,1))
X2_ = scaler.fit_transform(X_)

#Dividimos data
X2_train,X2_test,Y_train,Y_test = tts(X2_,Y_,test_size = 0.40,random_state=42)


#print(weather_tra)

#Inicio operaciones
estado=usuario_dec()
while True:
  if estado==1:
    print("Usted puede: Solicitar análisis estadístico, solicitar gráficos estadísticos y analíticos, entrenar el modelo u obtener el valor de predicción para determinado dato.")
    operaciones=input("¿Qué desea hacer? ")
    operaciones_lower = operaciones.lower()

     #Análisis estadístico
    if 'análisis' in operaciones_lower:
      dec_=input('¿De qué columna desea su análisis estadístico? ')
      dec_g=dec_.lower()
      print(weather_df[dec_g].describe())
      estado=usuario_dec()
      
      print('')

     #Gráficos   
    elif 'gráficos' in operaciones_lower:
      print('Usted puede graficar: Gráfico de dispersión(GD), Histograma(H), Gráfico de violín(GV), Gráfico de caja(GC), Gráfico de pastel(GP) o Gráfico de línea(GL)')
      decgraficar=input('¿Qué desea graficar? ')

#Histograma
      if decgraficar.lower()=='histograma' or decgraficar.lower()=='h':
        dec_hist=input('Quieres un histograma de: ')
        fighist = px.histogram(weather_df,x=dec_hist,nbins=38,width=400,height=300)
        fighist.show()

#Gráfico de dispersión
      elif decgraficar.lower()=='gráfico de dispersión' or decgraficar.lower()=='gd':
        xdis=input('Escoga su x: ')
        ydis=input('Escoga su y: ')
        figdis = px.scatter(data_frame=weather_df , x= xdis, y= ydis, color='weather',width=600,height=450)
        figdis.show()
      
#Gráfico de pastel
      elif decgraficar.lower()=='gráfico de pastel' or decgraficar.lower()=='gp':
        figpastel=px.pie(weather_df,names="weather",width=600,height=450)
        figpastel.show()
      
#Gráfico de línea
      elif decgraficar.lower()=='gráfico de línea' or decgraficar.lower()=='gl':
        rry=input('Escoga su y: ')
        figlinea=px.line(weather_df,x="date",y=rry,color="weather")
        figlinea.show()

#Gráfico de caja       
      elif decgraficar.lower()=='gráfico de caja' or decgraficar.lower()=='gc':
        yddis=input('Escoga su y: ')
        figcaja = px.box(weather_df,x="weather",y=yddis,points="all",width=680,height=570)
        figcaja.show()

#Gráfico de violín
      elif decgraficar.lower()=='gráfico de violín' or decgraficar.lower()=='gv':
        ydiss=input('Escoga su y: ')
        figviol = px.violin(weather_df,x="weather",y=ydiss,points="all",width=680,height=570)
        figviol.show()
      print('')
      estado=usuario_dec()

#Entrenamiento del modelo
    elif 'entrenar' in operaciones_lower:
      print('Usted puede entrenar el modelo con: KNN, Random Forest(RF), Suport Vector Machine(SVM) o Gradient Boosting(GB)')
      tipo=input('Escoga el algoritmo: ')
      print('')

#KNN
      if tipo.lower()=='knn':
        modeloKNN = KNc(n_neighbors=3)
        modeloKNN.fit(X2_train,Y_train)

        Y_predi1 = modeloKNN.predict(X2_test) 
        Y_predtrain = modeloKNN.predict(X2_train)
        score_totaltrain = modeloKNN.score(X2_train,Y_train)
        recallscoretrain  = recall_score(Y_train,Y_predtrain,average=None)
        precisionscoretrain = precision_score(Y_train,Y_predtrain,average=None)
        print('')
        print('Métricas entrenamiento')
        print("El score de entrenamiento es",score_totaltrain)
        print("El recall de entrenamiento es",recallscoretrain)
        print("La precision de entrenamiento es",precisionscoretrain)
        print('')

        print('Métricas validación')
        score_totalvalidation = modeloKNN.score(X2_test,Y_test)
        recallscoreval  = recall_score(Y_test,Y_predi1,average=None)
        precisionscoreval = precision_score(Y_test,Y_predi1,average=None)
        print("El score de validación es",score_totalvalidation)
        print("El recall de validación es",recallscoreval)
        print("La precision de validación es",precisionscoreval)

#Random Forest
      elif tipo.lower()=='random forest' or tipo.lower()=='rf':
        modeloRF=RFc(random_state=42)
        modeloRF.fit(X2_train,Y_train)

        Y_predi1 = modeloRF.predict(X2_test) 
        Y_predtrain = modeloRF.predict(X2_train)
        score_totaltrain = modeloRF.score(X2_train,Y_train)
        recallscoretrain  = recall_score(Y_train,Y_predtrain,average=None)
        precisionscoretrain = precision_score(Y_train,Y_predtrain,average=None)
        print('')
        print('Métricas entrenamiento')
        print("El score de entrenamiento es",score_totaltrain)
        print("El recall de entrenamiento es",recallscoretrain)
        print("La precision de entrenamiento es",precisionscoretrain)
        print('')

        print('Métricas validación')
        score_totalvalidation = modeloRF.score(X2_test,Y_test)
        recallscoreval  = recall_score(Y_test,Y_predi1,average=None)
        precisionscoreval = precision_score(Y_test,Y_predi1,average=None)
        print("El score de validación es",score_totalvalidation)
        print("El recall de validación es",recallscoreval)
        print("La precision de validación es",precisionscoreval)

#Gradient Boosting
      elif tipo.lower()=='gradient boosting' or tipo.lower()=='gb':
        modeloGB=GBC(learning_rate=0.1,max_depth=3)
        modeloGB.fit(X2_train,Y_train)

        Y_predi1 = modeloGB.predict(X2_test) 
        Y_predtrain = modeloGB.predict(X2_train)
        score_totaltrain = modeloGB.score(X2_train,Y_train)
        recallscoretrain  = recall_score(Y_train,Y_predtrain,average=None)
        precisionscoretrain = precision_score(Y_train,Y_predtrain,average=None)
        print('')
        print('Métricas entrenamiento')
        print("El score de entrenamiento es",score_totaltrain)
        print("El recall de entrenamiento es",recallscoretrain)
        print("La precision de entrenamiento es",precisionscoretrain)
        print('')

        print('Métricas validación')
        score_totalvalidation = modeloGB.score(X2_test,Y_test)
        recallscoreval  = recall_score(Y_test,Y_predi1,average=None)
        precisionscoreval = precision_score(Y_test,Y_predi1,average=None)
        print("El score de validación es",score_totalvalidation)
        print("El recall de validación es",recallscoreval)
        print("La precision de validación es",precisionscoreval)

#Suport Vector Machine
      elif tipo.lower()=='suport vector machine' or tipo.lower()=='svm':
        modeloSVM=svc(kernel='rbf',gamma='scale')
        modeloSVM.fit(X2_train,Y_train)

        Y_predi1 = modeloSVM.predict(X2_test) 
        Y_predtrain = modeloSVM.predict(X2_train)
        score_totaltrain = modeloSVM.score(X2_train,Y_train)
        recallscoretrain  = recall_score(Y_train,Y_predtrain,average=None)
        precisionscoretrain = precision_score(Y_train,Y_predtrain,average=None)
        print('')
        print('Métricas entrenamiento')
        print("El score de entrenamiento es",score_totaltrain)
        print("El recall de entrenamiento es",recallscoretrain)
        print("La precision de entrenamiento es",precisionscoretrain)
        print('')

        print('Métricas validación')
        score_totalvalidation = modeloSVM.score(X2_test,Y_test)
        recallscoreval  = recall_score(Y_test,Y_predi1,average=None)
        precisionscoreval = precision_score(Y_test,Y_predi1,average=None)
        print("El score de validación es",score_totalvalidation)
        print("El recall de validación es",recallscoreval)
        print("La precision de validación es",precisionscoreval)
      print('')
      estado=usuario_dec()

#Valor de predicción
    elif "predecir" in operaciones_lower:
      al=float(input('Ingresa el valor de precipitation: '))
      bl=float(input('Ingresa el valor de temp_max: '))
      cl=float(input('Ingresa el valor de temp_min: '))
      dl=float(input('Ingresa el valor de wind: '))
      x_df=weather_df.drop(columns=["weather","date"])
      x_max=x_df.max()
      x_min=x_df.min()
      numb=0
      arr=[]
      for i in [al,bl,cl,dl]:
        arr.append(1-((x_max[numb]-i)/(x_max[numb]-x_min[numb])))
        numb=numb+1
      arr=np.array([arr])
      entrada_=arr
      #print(entrada_)

      pre_dec=input('¿Con qué algoritmo desea predecir? ')

      if pre_dec.lower()=='knn':
        res=modeloKNN.predict(entrada_)
        print(res)
        if res==0:
          print('Llovizna')
        elif res==1:
          print('Día de niebla')
        elif res==2:
          print('Día lluvioso')
        elif res==3:
          print('Día nevado')
        else:
          print('Día soleado')

      elif pre_dec.lower()=='svm' or pre_dec.lower()=='suport vector machine':
        ress=modeloSVM.predict(entrada_)
        if ress==0:
          print('Llovizna')
        elif ress==1:
          print('Día de niebla')
        elif ress==2:
          print('Día lluvioso')
        elif ress==3:
          print('Día nevado')
        else:
          print('Día soleado')

      elif pre_dec.lower()=='gb' or pre_dec.lower()=='gradient boosting':
        resss=modeloGB.predict(entrada_)
        if resss==0:
          print('Llovizna')
        elif resss==1:
          print('Día de niebla')
        elif resss==2:
          print('Día lluvioso')
        elif resss==3:
          print('Día nevado')
        else:
          print('Día soleado')

      elif pre_dec.lower()=='rf' or pre_dec.lower()=='random forest':
        reesss=modeloRF.predict(entrada_)
        if reesss==0:
          print('Llovizna')
        elif reesss==1:
          print('Día de niebla')
        elif reesss==2:
          print('Día lluvioso')
        elif reesss==3:
          print('Día nevado')
        else:
          print('Día soleado')


      print('')
      estado=usuario_dec()



    else:
      break

  if estado==2:
    print("Cerrar")
    break

print('Programa cerrado')


#


            date  precipitation  temp_max  temp_min  wind  weather
0     2012-01-01            0.0      12.8       5.0   4.7  drizzle
1     2012-01-02           10.9      10.6       2.8   4.5     rain
2     2012-01-03            0.8      11.7       7.2   2.3     rain
3     2012-01-04           20.3      12.2       5.6   4.7     rain
4     2012-01-05            1.3       8.9       2.8   6.1     rain
...          ...            ...       ...       ...   ...      ...
1456  2015-12-27            8.6       4.4       1.7   2.9     rain
1457  2015-12-28            1.5       5.0       1.7   1.3     rain
1458  2015-12-29            0.0       7.2       0.6   2.6      fog
1459  2015-12-30            0.0       5.6      -1.0   3.4      sun
1460  2015-12-31            0.0       5.6      -2.1   3.5      sun

[1461 rows x 6 columns]

Si desea hacer una acción escriba W: W
Usted puede: Solicitar análisis estadístico, solicitar gráficos estadísticos y analíticos, entrenar el modelo u obtener el valor de

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))




Métricas entrenamiento
El score de entrenamiento es 0.7545662100456622
El recall de entrenamiento es [0.         0.         0.80052493 0.125      0.91237113]
La precision de entrenamiento es [0.         0.         0.84022039 1.         0.6927593 ]

Métricas validación
El score de validación es 0.7623931623931623
El recall de validación es [0.         0.         0.81923077 0.         0.92460317]
La precision de validación es [0.         0.         0.86234818 0.         0.68934911]

Si desea hacer una acción escriba W: W
Usted puede: Solicitar análisis estadístico, solicitar gráficos estadísticos y analíticos, entrenar el modelo u obtener el valor de predicción para determinado dato.
¿Qué desea hacer? PREDECIR
Ingresa el valor de precipitation: 14.25
Ingresa el valor de temp_max: 10.24
Ingresa el valor de temp_min: 4.2
Ingresa el valor de wind: 2.05
¿Con qué algoritmo desea predecir? SVM
Día lluvioso

Si desea hacer una acción escriba W: W
Usted puede: Solicitar análisis estadístico, s

/usr/local/lib/python3.7/dist-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)




Métricas entrenamiento
El score de entrenamiento es 0.8105022831050228
El recall de entrenamiento es [0.47222222 0.52727273 0.86351706 0.4375     0.84536082]
La precision de entrenamiento es [0.37777778 0.48333333 0.8844086  1.         0.83673469]

Métricas validación
El score de validación es 0.6974358974358974
El recall de validación es [0.11764706 0.15217391 0.80384615 0.2        0.74603175]
La precision de validación es [0.07692308 0.15909091 0.83935743 1.         0.71212121]

Si desea hacer una acción escriba W: W
Usted puede: Solicitar análisis estadístico, solicitar gráficos estadísticos y analíticos, entrenar el modelo u obtener el valor de predicción para determinado dato.
¿Qué desea hacer? PREDECIR
Ingresa el valor de precipitation: 18.2
Ingresa el valor de temp_max: 4.2
Ingresa el valor de temp_min: 2
Ingresa el valor de wind: 2
¿Con qué algoritmo desea predecir? KNN
[2]
Día lluvioso

Si desea hacer una acción escriba W: W
Usted puede: Solicitar análisis estadístico, solic

/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)



Métricas entrenamiento
El score de entrenamiento es 0.9292237442922374
El recall de entrenamiento es [0.41666667 0.52727273 0.96062992 1.         1.        ]
La precision de entrenamiento es [1.         1.         1.         1.         0.86222222]

Métricas validación
El score de validación es 0.8324786324786325
El recall de validación es [0.05882353 0.08695652 0.91538462 0.4        0.95238095]
La precision de validación es [0.5        0.30769231 0.96356275 0.4        0.76677316]

Si desea hacer una acción escriba W: W
Usted puede: Solicitar análisis estadístico, solicitar gráficos estadísticos y analíticos, entrenar el modelo u obtener el valor de predicción para determinado dato.
¿Qué desea hacer? PREDECIR
